## The objective of this notebook is to Predict the Total Fare for the ride

In [1]:
import pandas as pd

Loading dataset

In [2]:
df=pd.read_csv('./taxi_fare/train.csv')
df.head()

,trip_duration,distance_traveled,num_of_passengers,fare,tip,miscellaneous_fees,total_fare,surge_applied
0,748.0,2.75,1.0,75.00,24,6.300,105.300,0
1,1187.0,3.43,1.0,105.00,24,13.200,142.200,0
2,730.0,3.12,1.0,71.25,0,26.625,97.875,1
3,671.0,5.63,3.0,90.00,0,9.750,99.750,0
4,329.0,2.09,1.0,45.00,12,13.200,70.200,0


In [3]:
df.describe()

,trip_duration,distance_traveled,num_of_passengers,fare,tip,miscellaneous_fees,total_fare,surge_applied
count,209673.000000,209673.000000,209673.000000,209673.000000,209673.000000,209673.000000,209673.000000,209673.000000
mean,1173.181478,5.054431,1.292808,99.623431,13.030824,15.136829,127.791084,0.280523
std,4775.653621,125.217419,0.930754,85.602702,20.367764,12.553436,98.797374,0.449256
min,0.000000,0.020000,0.000000,0.000000,0.000000,-0.500000,0.000000,0.000000
25%,446.000000,1.950000,1.000000,52.500000,0.000000,6.000000,70.200000,0.000000
50%,707.000000,3.200000,1.000000,75.000000,9.000000,9.750000,101.700000,0.000000
75%,1098.000000,5.730000,1.000000,116.250000,20.000000,26.450000,152.250000,1.000000
max,86387.000000,57283.910000,9.000000,4466.250000,2500.000000,435.000000,4472.250000,1.000000


In [4]:
df.shape

(209673, 8)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209673 entries, 0 to 209672
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   trip_duration       209673 non-null  float64
 1   distance_traveled   209673 non-null  float64
 2   num_of_passengers   209673 non-null  float64
 3   fare                209673 non-null  float64
 4   tip                 209673 non-null  int64  
 5   miscellaneous_fees  209673 non-null  float64
 6   total_fare          209673 non-null  float64
 7   surge_applied       209673 non-null  int64  
dtypes: float64(6), int64(2)
memory usage: 12.8 MB


Verifying if there are null values

In [11]:
df.isna().sum()

trip_duration         0
distance_traveled     0
num_of_passengers     0
fare                  0
tip                   0
miscellaneous_fees    0
total_fare            0
surge_applied         0
dtype: int64

In [7]:
df.duplicated().sum()

4325

In [8]:
df[df.duplicated(['trip_duration'])==True].head()

,trip_duration,distance_traveled,num_of_passengers,fare,tip,miscellaneous_fees,total_fare,surge_applied
48,730.0,3.35,1.0,75.00,22,33.950,130.950,1
54,453.0,2.12,1.0,52.50,0,6.000,58.500,0
82,319.0,1.32,1.0,41.25,0,9.750,51.000,0
88,234.0,1.22,1.0,33.75,0,6.000,39.750,0
92,1592.0,5.66,1.0,131.25,0,26.625,157.875,1


In [9]:
df[df['trip_duration']==730].head()

,trip_duration,distance_traveled,num_of_passengers,fare,tip,miscellaneous_fees,total_fare,surge_applied
2,730.0,3.12,1.0,71.25,0,26.625,97.875,1
48,730.0,3.35,1.0,75.00,22,33.950,130.950,1
393,730.0,2.30,1.0,60.00,0,6.000,66.000,0
1991,730.0,2.93,6.0,71.25,26,34.450,131.700,1
6065,730.0,4.18,1.0,78.75,22,14.000,114.750,0


Even if there are duplicated values, such as 'trip_duration', 'fare', 'tip' and so on, these values are important in order to measure how much a feature was repeated.

             total_fare=fare+tip+miscellaneous_fees

Getting the predictors and the prediction values

Independent Variables

In [15]:
X=df.drop(columns='total_fare')

In [17]:
X.head()

,trip_duration,distance_traveled,num_of_passengers,fare,tip,miscellaneous_fees,surge_applied
0,748.0,2.75,1.0,75.00,24,6.300,0
1,1187.0,3.43,1.0,105.00,24,13.200,0
2,730.0,3.12,1.0,71.25,0,26.625,1
3,671.0,5.63,3.0,90.00,0,9.750,0
4,329.0,2.09,1.0,45.00,12,13.200,0


Dependent Variable

In [19]:
Y=df['total_fare']

In [21]:
Y.head()

0    105.300
1    142.200
2     97.875
3     99.750
4     70.200
Name: total_fare, dtype: float64